In [1]:
import pandas as pd
import numpy as np
from nn import DynamicANNWrapper, DynamicANN
from sklearn.model_selection import train_test_split
import torch
from wrapper_model import wrapper_model

from sklearn.metrics import accuracy_score

/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'si'

In [ ]:
df = pd.read_csv('prepro_jung.csv')


# 타겟값 분리

In [ ]:

# 판다스 데이터프레임을 트레인과 테스트로 나누기
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2023)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2023)

# 여기서
# - your_dataframe은 나눌 데이터프레임을 대체하는 변수명입니다.
# - test_size는 테스트 셋의 비율을 나타냅니다. 0.2는 80% 트레인, 20% 테스트로 나누겠다는 뜻입니다.
# - random_state는 나누는 과정에서 사용되는 난수 발생 시드(seed)를 의미하며, 동일한 결과를 얻기 위해 설정합니다.

# 결과로 나온 train_df와 test_df를 사용하여 모델을 훈련하고 테스트할 수 있습니다.

In [ ]:
train_df.info()


In [ ]:
X_trn = train_df.drop('타겟',axis=1)
y_trn = train_df['타겟']
X_tst = test_df.drop('타겟',axis=1)
y_tst = test_df['타겟']


In [ ]:
X_trn = X_trn.values
y_trn = y_trn.values
X_tst = X_tst.values

In [ ]:
X_trn.shape, y_trn.shape, X_tst.shape

In [ ]:
models = wrapper_model(X_trn, y_trn, X_tst ,'config.json')

In [ ]:
model = models[0]
print(model.score(y_tst, X_tst))

In [ ]:
feature = val_df.drop('타겟',axis=1)
label = val_df['타겟']

feature = feature.values
label = label.values

In [ ]:
performance_metrics = CV(models, feature, label, n_splits=5)